In [ ]:
!pip install fuzzywuzzy -qqq
!pip install natasha -qqq

In [1]:
import tqdm 
import pickle
import string
import re
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

from nltk.metrics.distance import jaro_similarity 
import random

In [13]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)


def lemmatize_data(text):
    doc = Doc(text)
    
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        
    lemma_dict = " ".join([_.lemma for _ in doc.tokens])
    return lemma_dict



def test():
    text = 'Аденостилес ромболистный'
    lemma_dict = lemmatize_data(text)
    return lemma_dict

test()

'аденостилес ромболистный'

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/хакатон/plants_base.pkl", 'rb') as f:
    data = pickle.load(f)

In [ ]:
def format_entity(entity_str):
    entity_str = entity_str.lower()
    entity_str = re.sub(r'\([\s\S]*\)', "", entity_str).strip()
    entity_str = entity_str.translate(str.maketrans('', '', string.punctuation))
    return entity_str

In [ ]:
pars_list_plans = [[x.lower(), x] for x in list(data.keys())]

In [ ]:
data_chim = []
name = []
for j in pars_list_plans:
    data_chim.append(data[j[1]]["Химический"])
    name.append(j[1])
    if len(data_chim[-1])==0:
        data_chim[-1] = ['']


In [ ]:
data_chim = list(map(lambda x: lemmatize_data(' '.join(x)), data_chim))

In [621]:
with open("chim_components.json", "r") as outfile:
    chim_components = json.load(outfile)

In [622]:
chim_components = list(set(chim_components))

In [619]:
new_chim_components = []
for i in chim_components:
    if "+" in i:
        new_chim_components = new_chim_components + i.split('+')
    else:
        new_chim_components.append(i)

In [367]:
chim_components = []
for c_c in new_chim_components:
    chim_components.append(lemmatize_data(c_c))

In [384]:
new_chim_components = []

for i in chim_components:
    if len(i.split()) <3:
        new_chim_components.append(i)

In [ ]:
count = 0
for i in data_chim[0].split():
    for j in chim_components:
        if jaro_similarity(i, j)>=0.65:
            count+=1
            break

In [571]:
dict_chim_res = dict([])

for i,j in zip(name, data_chim):
    dict_chim_res[format_entity(i)] = dict({"chemical_composition": set()})
    for com in new_chim_components:
        for ch in j:
            if fuzz.ratio(com, ch)>=10:
                dict_chim_res[format_entity(i)]["chemical_composition"].add(com)
                break

In [573]:
for i,j in zip(name, data_chim):
    dict_chim_res[format_entity(i)] = dict({"chemical_composition": list(dict_chim_res[format_entity(i)]['chemical_composition']) })

In [ ]:
data_pochva = []
name_pochva = []
for j in pars_list_plans:
    data_pochva.append(data[j[1]]["Экология. "])
    name_pochva.append(j[1])
    if len(data_pochva[-1])==0:
        data_pochva[-1] = [""]


In [ ]:
len(name_pochva)

196

In [ ]:
data_pochva = list(map(lambda x: lemmatize_data(' '.join(x)), data_pochva))

In [ ]:
poch = []
last = ''
for i in data_pochva:
    for j in i.split():
        if "почв" in j:
            if len(last) >  5:
                poch.append(last)
        last=j

In [ ]:
dict_pochva_res = dict([])

for i,j in zip(name_pochva, data_pochva):
    dict_pochva_res[format_entity(i)] = set([])
    for com in poch:
        if com in j:
            dict_pochva_res[format_entity(i)].add(com)
    dict_pochva_res[format_entity(i)] = dict({"soil": list(dict_pochva_res[format_entity(i)]) })

In [586]:
with open("chim.json", "w") as outfile:
    json.dump(dict_chim_res, outfile, ensure_ascii=False, indent=4)

with open("pochva.json", "w") as outfile:
    json.dump(dict_pochva_res, outfile, ensure_ascii=False, indent=4)

In [590]:
with open("full_features_dict.json", "r", encoding="utf-8") as outfile:
    temp = json.loads(outfile.read())

In [596]:
name_plants = list(temp.keys())
fitch_name = set()

In [604]:
for i in name_plants:
    fitch_name = fitch_name | set(temp[i].keys())

In [605]:
fitch_name

{'chemicals_feature',
 'climate_feature',
 'location_feature',
 'pochva_feature',
 'red_book_feature'}

In [606]:
dict_fitch = {i:set() for i in fitch_name}

In [608]:
for i in name_plants:
    for j in fitch_name:
        dict_fitch[j] = dict_fitch[j] | set(temp[i][j])

In [613]:
for i in dict_fitch.keys():
    dict_fitch[i] = list(dict_fitch[i])

In [614]:
with open("fich_set.json", "w") as outfile:
    json.dump(dict_fitch, outfile, ensure_ascii=False, indent=4)

In [625]:
with open("SRMP_formatted.json", "r", encoding="utf-8") as outfile:
    medicines = json.loads(outfile.read())

In [10]:
medicines = pd.read_json("SRMP_formatted.json")

In [11]:
 
with open("chim.json", "r", encoding="utf-8") as outfile:
    dict_chim_res = json.loads(outfile.read())

In [14]:
medicines_new = []

for i,j in medicines.iterrows():
    for new in j.chemical_name.split("+"):
        if len(new.split()) <3:
            medicines_new.append([j.market_name, lemmatize_data(new)])

In [15]:
medicines_new = pd.DataFrame(medicines_new, columns=medicines.columns)

In [16]:
medicines_new

,market_name,chemical_name
0,"1,3-Диэтилбензимидазолия трийодид","1,3-диэтилбензимидазолия трийодид"
1,1-ХЛОРМЕТИЛСИЛАТРАН,"1-хлорметил-2,8,9-триокса-5-аза-1-силабицикло ..."
2,1-ацетиламино-5-нитро-2-пропоксибензол,1-ацетиламино-5-нитро-2-пропоксибензол
3,2-(Диметиламино)этанол,деанол
4,2-Аллилоксиэтанол,аллилоксиэтанол
...,...,...
12828,эфлейра®,нетакимаб
12829,эффералган®,парацетамол
12830,эхинацея композитум сн,echinacea d 6
12831,ярина®,дроспиренон этинилэстрадиол


In [20]:
from tqdm.notebook import tqdm

In [38]:
dict_chim_res_medic = dict([])

for i in tqdm(dict_chim_res):
    dict_chim_res_medic[i] = set()
    for j in dict_chim_res[i]["chemical_composition"]:
        j = j.replace("[",'')
        dict_chim_res_medic[i]= dict_chim_res_medic[i] | set(medicines_new[ medicines_new.chemical_name.str.contains(j) ].market_name) 


  0%|          | 0/196 [00:00<?, ?it/s]

In [41]:
dict_chim_res_medic_count = dict([])

In [49]:
for i in tqdm(dict_chim_res_medic):
    dict_chim_res_medic[i] = list(dict_chim_res_medic[i])

  0%|          | 0/196 [00:00<?, ?it/s]

In [48]:
for i in tqdm(dict_chim_res_medic):
    dict_chim_res_medic_count[i] = len(dict_chim_res_medic[i])

  0%|          | 0/196 [00:00<?, ?it/s]

In [86]:
with open("medicines_counter.json", "w") as outfile:
    json.dump(dict_chim_res_medic_count, outfile, ensure_ascii=False, indent=4)

In [50]:
with open("dict_chim_res_medic.json", "w") as outfile:
    json.dump(dict_chim_res_medic, outfile, ensure_ascii=False, indent=4)